### 对比误差(继续训练）
1. 全局误差
2. 由训练点处的误差

In [1]:
item = 'random_10_64_3'

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import scipy.io as scio
import torch
import torch.nn as nn
from torchinfo import summary
from timeit import default_timer
import sys
sys.path.append("../scripts/")
sys.path.append("../run/")
import yaml
from utilities import *
from deeponet_2d import *
# from fourier_2d import *
# device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [3]:
with open( '../run/config.yml') as f:
    config = yaml.full_load(f)
config = config[item]

TRAIN_PATH = config['TRAIN_PATH']
TEST_PATH  = config['TEST_PATH']
save_mode  = config['save_mode']
save_step  = config['save_step']
n_out      = config['n_out'] # rhoxy,phsxy,rhoyx,phsyx
model_path = "../model/"+config['name']+ "_"+str(n_out) # save path and name of model
model_path_temp = "../temp/"+config['name']+"_"+ str(n_out)
log_path = "../Log/"+config['name']+"_"+str(n_out)+'.log'

ntrain = config['ntrain']
ntest  = config['ntest']

batch_size = config['batch_size']
learning_rate = 0.0001#config['learning_rate']

epochs = config['epochs']
step_size = config['step_size']
gamma = config['gamma']

modes = config['modes']
width = config['width']

s_train = config['s_train']
r_train = config['r_train']
s_test = config['s_test']
r_test = config['r_test']
layer_num = config['layer_num']
last_size = config['last_size'] 
act_fno   = config['act_fno']

layer_sizes = config['layer_sizes']+[s_train[0]*s_train[1]]
act_func  = config['act_func']
init_func = config['init_func']
    

patience = config['patience'] # if there is {patience} epoch that val_error is larger, early stop,
thre_epoch = config['thre_epoch']# condiser early stop after {thre_epoch} epochs

print_model_flag = config['print_model_flag'] # 

In [4]:
loc_train,loc_test,train_loader, test_loader, x_normalizer,y_normalizer = \
get_batch_data(TRAIN_PATH, TEST_PATH, ntrain, ntest, r_train, s_train,r_test, s_test,batch_size,n_out)
data   = scio.loadmat(TEST_PATH)
zn     = data['zn'][0][::r_test[0]][:s_test[0]+1]
yn     = data['yn'][0][::r_test[1]][:s_test[1]+1]
freq   = data['freq'][0][::r_test[2]][:s_test[2]]
ry     = data['obs'][0][::r_test[3]][:s_test[3]]

begin to read data
reading finished in 11.146 s


In [5]:
if os.path.exists(model_path+'.pt'):
    model=torch.load(model_path+'.pt',map_location=device)
elif os.path.exists(model_path+'.pkl'):
    model=deeponet(layer_sizes, act_func, init_func,modes, modes, width,\
        n_out,layer_num, last_size, act_fno).to(device)
    model.load_state_dict(torch.load(model_path+'.pkl',map_location=device))
else:
    raise RuntimeError('no model file')
myloss = LpLoss(size_average=False)

In [6]:


model.eval()
model.to(device)
y_normalizer.to(device)
test_l2 = 0.0
# sample = next(iter(test_loader))
t1 = default_timer()
loc_test = loc_test.to(device)
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)

        out = model(loc_test,x)#.reshape(batch_size, s[2],s[3],-1)
        input_size = s_test[2]*s_test[3]
        n_out = y.shape[-1]
        out = torch.cat(([out[:,i*input_size:(i+1)*input_size].reshape(batch_size,s_test[2],s_test[3],-1) \
                for i in range(n_out)]),-1)
        out = y_normalizer.decode(out)

        test_l2 += myloss(out.reshape(batch_size,-1), y.reshape(batch_size,-1)).item()

test_l2 /= ntest

t2 = default_timer()
print(t2-t1, test_l2)

36.22261477075517 0.004816661228736242


In [7]:
interval, num = 7, 10

In [8]:
# label data

model.eval()
model.to(device)
y_normalizer.to(device)
test_l2 = 0.0
# sample = next(iter(test_loader))
t1 = default_timer()
loc_test = loc_test.to(device)
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)

        out = model(loc_test,x)#.reshape(batch_size, s[2],s[3],-1)
        input_size = s_test[2]*s_test[3]
        n_out = y.shape[-1]
        out = torch.cat(([out[:,i*input_size:(i+1)*input_size].reshape(batch_size,s_test[2],s_test[3],-1) \
                for i in range(n_out)]),-1)
        out = y_normalizer.decode(out)

        test_l2 += myloss(out[:,::interval,:,:][:,:num,:,:].reshape(batch_size,-1),\
                            y[:,::interval,:,:][:,:num,:,:].reshape(batch_size,-1)).item()

test_l2 /= ntest

t2 = default_timer()
print(t2-t1, test_l2)

33.353832721710205 0.0040711781183878585


In [9]:
y[:,::7,:,:][:,:,:,:].shape

torch.Size([50, 10, 64, 4])

In [10]:
# inner interpretation

model.eval()
model.to(device)
y_normalizer.to(device)
test_l2 = 0.0
# sample = next(iter(test_loader))
t1 = default_timer()
loc_test = loc_test.to(device)
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)

        out = model(loc_test,x)#.reshape(batch_size, s[2],s[3],-1)
        input_size = s_test[2]*s_test[3]
        n_out = y.shape[-1]
        out = torch.cat(([out[:,i*input_size:(i+1)*input_size].reshape(batch_size,s_test[2],s_test[3],-1) \
                for i in range(n_out)]),-1)
        out = y_normalizer.decode(out)
        
        out0 = out[:,torch.arange(out.shape[1])%interval!=0,:,:]
        out1 = out0#[:,:,torch.arange(out0.shape[2])%interval!=0,:]
        
        y0 = y[:,torch.arange(y.shape[1])%interval!=0,:,:]
        y1 = y0#[:,:,torch.arange(y0.shape[2])%interval!=0,:]

        test_l2 += myloss(out1.reshape(batch_size,-1), y1.reshape(batch_size,-1)).item()

test_l2 /= ntest

t2 = default_timer()
print(t2-t1, test_l2)

35.33319640904665 0.004938789998491605
